# Transform Pandas DF to Verseagility-JSON

- Give a pandas dataframe as input with the columns "label" and "text" to bring it in the correct JSON-format
- Just keep the other dummy variables as they are to make sure the data stays valid. You can just remove the question/answering component from your case file, so that this part does not get trained
- You can also change the values for url, date etc. if you have a respective column in your dataset

In [ ]:
# Import packages
import pandas as pd
import uuid
import json

In [ ]:
# Change this to your format respectively
df = pd.read_csv("file.csv", sep=";", encoding="utf-8")

In [ ]:
# Define the data transformer function
def transformJSON(df, language):
    with open(f"output-{language}-train.json", "a", encoding='utf-8') as file:
        file.write("[")
    for index, row in df.iterrows():
        fileid = uuid.uuid4().hex
        q_views = 0
        q_tags = row['label']
        url = "https://aka.ms/nlp-demo"
        lang = language

        # PACK Q JSON
        question = {}
        question['title'] = "Text"
        question['author'] = "Author"
        question['createdAt'] = "01 January, 2020"
        question['text'] = row['text']
        question['upvotes'] = 0

        # PACK A JSON
        answer = {}
        answer['markedAsAnswer'] = "false"
        answer['createdAt'] = "01 January, 2020"
        answer['text'] = "Text"
        answer['upvotes'] = 0

        # PACK JSON
        data = {'question': question, 'id': fileid, 'views': q_views, 'appliesTo': q_tags, 'url': url, 'language': lang, 'answer': answer}
        content = json.dumps(data, indent=4, separators=(',', ': '), ensure_ascii = False)

        # WRITE TO JSON FILE
        with open(f"output-{language}-train.json", "a", encoding='utf-8') as file:
            if index == len(df) - 1:
                file.write(content + "]")
            else:
                file.write(content + ",")
            print(f"[SUCCESS] - File {fileid}\n")

In [ ]:
# Initiate the transformation
transformJSON(df, "en-us")